# Testing experimentally bounding deviations from quantum theory in the landscape of generalized probabilistic theories

## Article

https://arxiv.org/abs/1710.05948

In [373]:
import numpy as np
import scipy as sp
from qiskit import *
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.providers.aer.extensions.snapshot_statevector import *
from math import pi, sqrt
from cvxopt import matrix, solvers
import quadprog

## Experiment

### First Test

In the first test we prepare $n=100$ preparations and $m=100$ measurement settings. First we create a function that generates $n$ equally distanced points or states on the surface of the bloch sphere.

In [374]:
def gen_eq_bloch_states(n, C=3.6, return_angles=True):
    '''
    Generates equally spaced points on the bloch sphere through a spiral. 
    See here: https://www.intlpress.com/site/pub/files/_fulltext/journals/mrl/1994/0001/0006/MRL-1994-0001-0006-a003.pdf

    ==========
    Parameters
    ----------
    n (type=int): the number of points to be generated evenly on the sphere
    C (type=float, default=3.6): constant that makes sure that succesive points 
                                 on S^2 will be the same Euclidean distance apart
    return_angles (type=boolean, default=True): whether to also return the angles 
                                                associated with each state
    ==========
    
    ======
    Return
    ------
    (type=numpy.array): an array for n evenly spaced coordinates on the block sphere, 
                         with their associated angles if needed
    ======
    '''
    states = np.zeros((n, 2), dtype=complex)
    h = np.zeros(n)
    theta = np.zeros(n)
    phi = np.zeros(n)
    
    # initialize
    h[0] = -1
    h[n-1] = -1 + 2*(n-1)/(n-1)
    theta[0] = np.arccos(h[0])
    theta[n-1] = np.arccos(h[n-1]) 
    phi[0] = 0
    phi[n-1] = 0
    states[0] = [np.cos(theta[0]/2), np.sin(theta[0]/2)*np.exp(1j*phi[0])]
    states[n-1] = [np.cos(theta[n-1]/2), np.sin(theta[n-1]/2)*np.exp(1j*phi[n-1])]
    
    # the rest of the states
    for k in range(1, n-1):
        h[k] = -1 + 2*(k)/(n-1)
        theta[k] = np.arccos(h[k])
        phi[k] = np.mod(phi[k] + C/(sqrt(n)*sqrt(1 - h[k]**2)), 2*pi)
        states[k] = [np.cos(theta[k]/2), np.sin(theta[k]/2)*np.exp(1j*phi[k])]
    
    if return_angles==True:
        return [states, theta, phi]
    else:
        return states

Next, we create circuits to simulate 100 preparations and 100 measurements. We will prepare 100 states of the form:

\\[\{|\psi_i>\}_{i=1}^{100}\\]

These states are uniformly distributed on the bloch sphere as implemented by the function above.

We will then implement the measurements with the following projective measurements:

\\[\{(|\psi_i><\psi_i|, \mathbb{I}-|\psi_i><\psi_i|)\}_{i=1}^{100}\\]

In [483]:
# get backend
backend = Aer.get_backend("aer_simulator")

# get list of n*m evenly distributed states on the bloch sphere
n = 100 # number of preparations
m = 100 # number of measurements
states_info = gen_eq_bloch_states(n)
F = np.zeros((n, m), dtype=float) # frequency matrix
var = np.zeros((n, m), dtype=float) # variance matrix

# loop through the preparations and measurements
for i in range(n):
    for j in range(m):
        qc = QuantumCircuit(1) # single qubit circuit
        state = states_info[0][i] #if j != 0 else states_info[0][0]
        
        # angles
        phi_j = states_info[2][j] if j != 0 else states_info[2][i]
        theta_j = states_info[1][j] if j != 0 else states_info[1][i]
        x = np.sin(theta_j)*np.cos(phi_j)
        y = np.sin(theta_j)*np.sin(phi_j)
        z = np.cos(theta_j)
        alpha_xz = np.arctan(x/z) if (x != 0 and y != 0 and theta_j != 0) else theta_j
        alpha_yz = np.arctan(y/z)
        
        # construct circuits
        qc.initialize(state)
        qc.barrier()
        qc.rz(-2*phi_j, 0)
        qc.rx(-theta_j, 0)
        qc.rz(phi_j, 0)
        qc.snapshot_statevector('label')
        qc.measure_all()
        
        # Transpile for simulator
        backend = Aer.get_backend('aer_simulator')
        qc = transpile(qc, backend)

        # Run and get counts
        runs = 1000
        result = backend.run(qc, shots=runs).result()
        counts = result.get_counts(qc)
        #print(result.data()['snapshots'])
        #print(counts)
        
        # calculate frequency matrix and variance matrix
        zero_meas = counts.get('0') if not counts.get('0') is None else 0
        F[i][j] = zero_meas/runs
        avg = (runs - zero_meas)/runs
        var[i][j] = 1/(runs - 1)*(zero_meas*(avg**2))*((runs - zero_meas)*((1 - avg)**2))

In [484]:
W = np.zeros((n*m, m*n), dtype=float) # W = 1/var
index = 0

for i in range(n):
    for j in range(m):
        W[index][index] = 1/(var[i][j]) if var[i][j] != 0 else 0
    index += 1

print(F)
print(states_info)

[[1.    0.984 0.977 ... 0.022 0.005 0.   ]
 [1.    0.999 0.988 ... 0.043 0.042 0.005]
 [0.999 0.993 1.    ... 0.071 0.068 0.022]
 ...
 [1.    0.052 0.073 ... 0.997 0.99  0.985]
 [0.999 0.039 0.053 ... 0.988 1.    0.99 ]
 [1.    0.009 0.023 ... 0.976 0.989 1.   ]]
[array([[ 6.12323400e-17+0.j        ,  1.00000000e+00+0.j        ],
       [ 1.00503782e-01+0.j        , -2.26140676e-01+0.96889596j],
       [ 1.42133811e-01+0.j        ,  2.84371879e-01+0.94811951j],
       [ 1.74077656e-01+0.j        ,  4.89928338e-01+0.85420559j],
       [ 2.01007563e-01+0.j        ,  5.98006824e-01+0.77587615j],
       [ 2.24733287e-01+0.j        ,  6.63361614e-01+0.71375508j],
       [ 2.46182982e-01+0.j        ,  7.06268965e-01+0.66376057j],
       [ 2.65908012e-01+0.j        ,  7.35934645e-01+0.62265008j],
       [ 2.84267622e-01+0.j        ,  7.57134635e-01+0.58816585j],
       [ 3.01511345e-01+0.j        ,  7.72591565e-01+0.5587425j ],
       [ 3.17820863e-01+0.j        ,  7.83970439e-01+0.53327315j]

In [477]:
def vec(M):
    return M.reshape((-1, 1), order="F")

def mat(v, n, m, dty=float):
    M = np.zeros((n, m), dtype=dty)
    for i in range(n):
        for j in range(m):
            M[i][j] = v[n*j + i]
    return M

def is_pos_def(K):
    try:
        np.linalg.cholesky(K)
        return 1 
    except np.linalg.linalg.LinAlgError as err:
        if 'Matrix is not positive definite' in err.message:
            return 0
        else:
            raise 

def S_min(S, E, F, W, m):
    I_m = np.identity(m)
    A_1 = np.transpose(vec(S)) @ (np.kron(E, I_m)) @ W @ (np.kron(np.transpose(E), I_m)) @ vec(S)
    A_2 = 2 * np.transpose(vec(S)) @ (np.kron(E, I_m)) @ W @ vec(F)
    A = A_1 - A_2
    return A[0][0]

def E_min(E, S, F, W, n):
    I_n = np.identity(n)
    A_1 = np.transpose(vec(E)) @ np.transpose(np.kron(I_n, S)) @ W @ (np.kron(I_n, S)) @ vec(E)
    A_2 = 2 * np.transpose(vec(E)) @ np.transpose(np.kron(I_n, S)) @ W @ vec(F)
    A = A_1 - A_2
    return A[0][0]

In [478]:
def s_min(E, W, F, n, m):
    I_m = np.identity(m, dtype=float)
    P = matrix(2 * (np.kron(E, I_m)) @ W @ (np.kron(np.transpose(E), I_m)))
    print("S:" + str(is_pos_def(P)))
    q = matrix(-2 * np.kron(E, I_m) @ W @ vec(F))
    G_0 = -np.kron(np.transpose(E), I_m)
    G_1 = np.kron(np.transpose(E), I_m)
    G = matrix(np.concatenate([G_0, G_1]))
    h_0 = (np.zeros(n*m))
    h_1 = (np.ones(n*m))
    h = matrix(np.concatenate([h_0, h_1]))
    return solvers.qp(P, q, G, h)

def e_min(S, W, F, n, m):
    I_n = np.identity(n, dtype=float)
    P = matrix(2 * (np.transpose(np.kron(I_n, S))) @ W @ (np.kron(I_n, S)))
    print("E:" + str(is_pos_def(P)))
    q = matrix(-2 * np.transpose(np.kron(I_n, S)) @ W @ vec(F))
    G_0 = -np.kron(I_n, S)
    G_1 = np.kron(I_n, S)
    G = matrix(np.concatenate([G_0, G_1]))
    h_0 = (np.zeros(n*m))
    h_1 = (np.ones(n*m))
    h = matrix(np.concatenate([h_0, h_1]))
    return solvers.qp(P, q, G, h)

def chi_squared(S, E, F, W, n, m):
    sum = 0
    w_index = 0
    D = S @ E
    for i in range(n):
        for j in range(m):
            sum += ((F[i][j] - D[i][j]))**2 * W[w_index][w_index]
            w_index += 1
    return sum

In [479]:
def bfp(k, E_0, F, W, n, m, max_iterations=5000, convergence_threshold=10E-6):
    S = np.zeros((n, k), dtype=float)
    E = E_0
    chi_squared_prev = 0
    chi_squared_curr = 0
    iteration = 1
    while (True):
        chi_squared_prev = chi_squared_curr
        S = mat(s_min(E, W, F, n, m)['x'], n, k)
        #print(s_min(E, W, F, n, m)['x'])
        #print(S)
        E = mat(e_min(S, W, F, n, m)['x'], k, m)
        #print(E)
        chi_squared_curr = chi_squared(S, E, F, W, n, m)
        if (iteration == max_iterations or (chi_squared_curr -  chi_squared_prev) < convergence_threshold):
            break
        iteration += 1
    return [S, E]

In [480]:
U, s, Vh = np.linalg.svd(F)
S = np.diag(s)
S_0 = U
E_0 = S @ Vh 

candidate_k = {2, 3, 4, 5}
for k in candidate_k:
    S_k = np.diag(s[0:k])
    Vh_k = Vh[0:k, 0:len(Vh[0])]
    E_0 = S_k @ Vh_k
    print(bfp(k, E_0, F, W, n, m)[1])

AttributeError: 'LinAlgError' object has no attribute 'message'

In [ ]:
def aic(k, chi_squared_k, m, n):
    r_k = k*(m + n - k)
    return chi_squared_k + r_k